In [ ]:
import requests
import sys
import time

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
api= "45929254f44e4117aa634667a2c345b8"
upload_endpoint = "https://api.assemblyai.com/v2/upload"
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"
filename = "/content/drive/MyDrive/Speech_Recognation/Basic/01-basics_output.wav"

headers = {"authorization" : api}

In [ ]:
#Upload
def upload(filename):
    def read_file(filename, chunk_size = 5242880):
        with open(filename,"rb") as _file:
            while True:
                data = _file.read(chunk_size)
                if not data:
                    break
                yield data

    upload_response = requests.post(
                            upload_endpoint,
                            headers = headers,
                            data = read_file(filename))
    
    print(upload_response.json())
    audio_url = upload_response.json()["upload_url"]

    return audio_url

audio_url = upload(filename)
audio_url

{'upload_url': 'https://cdn.assemblyai.com/upload/ebd6052f-aa21-48c2-8243-370a1c5cc2ae'}


'https://cdn.assemblyai.com/upload/ebd6052f-aa21-48c2-8243-370a1c5cc2ae'

In [ ]:
# Transcribe
def transcribe(audio_url):
    transcrpit_request = {"audio_url" : audio_url}
    transcript_response = requests.post(transcript_endpoint,json = transcrpit_request, headers = headers)
    job_id = transcript_response.json()["id"]

    print(transcript_response.json())
    return job_id

audio_url = upload(filename)
transcript_id = transcribe(audio_url)

{'upload_url': 'https://cdn.assemblyai.com/upload/b6b384d4-18a4-415d-9972-86da7f59da90'}
{'id': 'o37g77p0iw-e5f8-435e-ad70-ed5a9592cb23', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://cdn.assemblyai.com/upload/b6b384d4-18a4-415d-9972-86da7f59da90', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': False, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': False, 'content_safety_labels': {}, 'iab_catego

In [ ]:
# Poll
def poll(transcript_id):    
    polling_endpoint = transcript_endpoint + "/" + transcript_id
    polling_response = requests.get(polling_endpoint,headers = headers)
    
    print(polling_response)
    # Status important
    print(polling_response.json())
    print(polling_response.json()["status"])

    return polling_response.json()

print("".center(50,"-"))
print(poll(transcript_id))

<Response [200]>
{'id': 'o37g77p0iw-e5f8-435e-ad70-ed5a9592cb23', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://cdn.assemblyai.com/upload/b6b384d4-18a4-415d-9972-86da7f59da90', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': False, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': False, 'content_safety_labels': {}, 'iab_categories_result': {}, 'language_detection': False, 'custom_spelling': None, 

In [ ]:
def get_transcription_result_url(url):
    transcribe_id = transcribe(url)
    while True:
        data = poll(transcribe_id)
        ##polling_response = requests.get(polling_endpoint,headers = headers)
        if data["status"] == "completed":
            return data,None
        elif data["status"] == "error":
            return data,data["error"]

In [ ]:
audio_url=upload(filename)
data,error = get_transcription_result_url(audio_url)

print(data)
print("".center(50,"*"))
print(error)

{'upload_url': 'https://cdn.assemblyai.com/upload/75911dcc-28a4-4697-ac22-aa400321e79c'}
{'id': 'o37zqq66xc-4a7a-4011-9ef6-ba7493d8502b', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://cdn.assemblyai.com/upload/75911dcc-28a4-4697-ac22-aa400321e79c', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': False, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': False, 'content_safety_labels': {}, 'iab_catego

In [ ]:
def save_transcription(url,filename_text):
    data,error = get_transcription_result_url(url)

    if data:
        text_filename = filename_text + ".txt"
        with open(text_filename, "w") as file:
            file.write(data["text"])
        print("Transcriptiopn Saved !!!")
    elif error:
        print("Error!! ",error)

    print("Waiting 10 seconds....")
    time.sleep(10)
    print("Done!!!")

In [ ]:
example = "/content/drive/MyDrive/Speech_Recognation/Speech_Reg/02-simple-speech-recognition_Natural Language Processing Short.m4a"
audio_url = upload(example)
save_transcription(audio_url,"result")

## Result

In [ ]:
import requests
import time

api= "45929254f44e4117aa634667a2c345b8"
upload_endpoint = 'https://api.assemblyai.com/v2/upload'
transcript_endpoint = 'https://api.assemblyai.com/v2/transcript'
headers = {"authorization" : api}

def upload(filename):
    def read_file(filename,chunk_size = 5242880):
        with open(filename,"rb") as f:
            while True:
                data = f.read(chunk_size)
                if not data:
                    break
                yield data

    upload_response = requests.post(upload_endpoint,headers = headers, data = read_file(filename))
    return upload_response.json()["upload_url"]

def transcribe(audio_url):
    transcrpit_request = {
        "audio_url" : audio_url
    }
    transcript_response = requests.post(transcript_endpoint,headers = headers, json=transcrpit_request)
    return transcript_response.json()["id"]

def poll(transcript_id):
    polling_endpoint = transcript_endpoint + "/" + transcript_id
    polling_response = requests.get(polling_endpoint,headers = headers)
    return polling_response.json()

def get_transcription_result_url(url):
    transcribe_id=transcribe(url)
    while True:
        data = poll(transcribe_id)
        if data["status"] == "completed":
            return data,None
        elif data["status"] == "error":
            return data,data["error"]
    
    print("Waiting For 5 Seconds")
    time.sleep(5)

def save_transcrpit(url,title):
    data,error = get_transcription_result_url(url)

    if data:
        filename = title + ".txt"
        with open(filename, "w") as f:
            f.write(data["text"])
        print("Transcript Saved !!!")
    elif error:
        print("Error : " ,error)

    print("Done !!!")

example = "/content/drive/MyDrive/Speech_Recognation/Speech_Reg/02-simple-speech-recognition_Natural Language Processing Short.m4a"
audio_url = upload(example)        
save_transcrpit(audio_url,"example")

Transcript Saved !!!
Done !!!
